# *Script* de Inferência

## Instalação de dependências

In [1]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 12.5 MB/s eta 0:00:0000:0100:01


## Modelo

### Criação de pipeline para carregar o modelo de difusão para memória

In [2]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
from diffusers import DDIMScheduler

model_path = "/kaggle/input/logodiffusiondataset/logo_model_1000/new_model"  

pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=True,
        ),
        safety_checker=None
    )

pipe = pipe.to("cuda")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:115: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.16.1",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "sample_max_value": 1.0,
  "set_alpha_to_one": true,
  "steps_offset": 0,
  "thresholding": false,
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull request for the `scheduler/scheduler_config.json` file
  deprecate("steps_offset!=1

### Geração de imagens dada uma prompt

In [3]:
def generate_images(prompt):
    neg_prompt = "ugly, abstract, blur, text, letters, background, drawing, tiling, painting, out of frame, deformed, watermark, signature, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur"
    images = pipe(prompt, num_inference_steps=50, num_images_per_prompt=3, guidance_scale=15.0, negative_prompt=neg_prompt).images
    for i, image in enumerate(images):
        image.save(f"image_{i+1}.png")
        plt.subplot(1,3,i+1)
        plt.imshow(image)

### Parâmetros chave da prompt

In [4]:
def parse_prompt(prompt):
    return f"{prompt}, big margins, small, logo design, logo style, emblem, 2d, centered, modern, minimalism"

In [5]:
#generate_images(parse_prompt("a logo of a coffeeshop, a red mug, red foreground, white background"))

## Atendimento de pedidos

### Verifica se recebeu um pedido de criação de logótipo. Ao receber um pedido, gera o logótipo dada a descrição textual

In [ ]:
import requests
import time

SERVER_URL = "https://541a4409-4ad3-4b64-ad82-5d699e911861.deepnoteproject.com"
image_files = ['image_1.png', 'image_2.png', 'image_3.png']

while True:
    time.sleep(2)
    prompt = requests.get(SERVER_URL + "/prompt").text
    if prompt != "[NONE]" and not "<html" in prompt.lower():
        prompt = parse_prompt(prompt)
        print("[PROMPT]", prompt)
        generate_images(prompt)
        print("Generated")
        files = [('image', open(file, 'rb')) for file in image_files]
        requests.post(SERVER_URL + "/gen_images", files=files)

[PROMPT] a logo of a coffee shop, big margins, small, logo design, logo style, emblem, 2d, centered, modern, minimalism


  0%|          | 0/50 [00:00<?, ?it/s]

Generated
[PROMPT] a logo of a dog sitting with a white background, big margins, small, logo design, logo style, emblem, 2d, centered, modern, minimalism


  0%|          | 0/50 [00:00<?, ?it/s]

Generated
